# MNIST Image Classification with TensorFlow on Cloud ML Engine

This notebook demonstrates how to implement different image models on MNIST using Estimator. 

Note the MODEL_TYPE and change it to try out different models.

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
from datetime import datetime
import os

PROJECT = "qwiklabs-gcp-00-02c9d8dfaa4e"  # REPLACE WITH YOUR PROJECT ID
BUCKET = "qwiklabs-gcp-00-02c9d8dfaa4e"  # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1"  # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "dnn_dropout"  # "linear", "dnn_dropout", "cnn", or "dnn"

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "2.1"  # Tensorflow  version
os.environ["IMAGE_URI"] = os.path.join("gcr.io", PROJECT, "mnistmodel")

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Building a dynamic model

The boilerplate structure for this module has already been set up in the folder `mnistmodel`. The module lives in the sub-folder, `trainer`, and is designated as a python package with the empty `__init__.py` (`mnistmodel/trainer/__init__.py`) file. It still needs the model and a trainer to run it, so let's make them.

Let's start with the trainer file first. This file parses command line arguments to feed into the model.

In [4]:
%%writefile mnistmodel/trainer/task.py
import argparse
import json
import os
import sys

from . import model


def _parse_arguments(argv):
    """Parses command-line arguments."""
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--model_type',
        help='Which model type to use',
        type=str, default='linear')
    parser.add_argument(
        '--epochs',
        help='The number of epochs to train',
        type=int, default=10)
    parser.add_argument(
        '--steps_per_epoch',
        help='The number of steps per epoch to train',
        type=int, default=100)
    parser.add_argument(
        '--job-dir',
        help='Directory where to save the given model',
        type=str, default='mnistmodel/')
    return parser.parse_known_args(argv)


def main():
    """Parses command line arguments and kicks off model training."""
    args = _parse_arguments(sys.argv[1:])[0]

    # Configure path for hyperparameter tuning.
    trial_id = json.loads(
        os.environ.get('TF_CONFIG', '{}')).get('task', {}).get('trial', '')
    output_path = args.job_dir if not trial_id else args.job_dir + '/'

    model_layers = model.get_layers(args.model_type)
    image_model = model.build_model(model_layers, args.job_dir)
    model_history = model.train_and_evaluate(
        image_model, args.epochs, args.steps_per_epoch, args.job_dir)


if __name__ == '__main__':
    main()


Overwriting mnistmodel/trainer/task.py


Next, let's group non-model functions into a util file to keep the model file simple. We'll copy over the `scale` and `load_dataset` functions from the previous lab.

In [5]:
%%writefile mnistmodel/trainer/util.py
import tensorflow as tf


def scale(image, label):
    """Scales images from a 0-255 int range to a 0-1 float range"""
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.expand_dims(image, -1)
    return image, label


def load_dataset(
        data, training=True, buffer_size=5000, batch_size=100, nclasses=10):
    """Loads MNIST dataset into a tf.data.Dataset"""
    (x_train, y_train), (x_test, y_test) = data
    x = x_train if training else x_test
    y = y_train if training else y_test
    # One-hot encode the classes
    y = tf.keras.utils.to_categorical(y, nclasses)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(scale).batch(batch_size)
    if training:
        dataset = dataset.shuffle(buffer_size).repeat()
    return dataset


Writing mnistmodel/trainer/util.py


Finally, let's code the models! The [tf.keras API](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras) accepts an array of [layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers) into a [model object](https://www.tensorflow.org/api_docs/python/tf/keras/Model), so we can create a dictionary of layers based on the different model types we want to use. The below file has two functions: `get_layers` and `create_and_train_model`. We will build the structure of our model in `get_layers`. Last but not least, we'll copy over the training code from the previous lab into `train_and_evaluate`.

These models progressively build on each other. Look at the imported `tensorflow.keras.layers` modules and the default values for the variables defined in `get_layers` for guidance.

In [6]:
%%writefile mnistmodel/trainer/model.py
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import (
    Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Softmax)

from . import util


# Image Variables
WIDTH = 28
HEIGHT = 28


def get_layers(
        model_type,
        nclasses=10,
        hidden_layer_1_neurons=400,
        hidden_layer_2_neurons=100,
        dropout_rate=0.25,
        num_filters_1=64,
        kernel_size_1=3,
        pooling_size_1=2,
        num_filters_2=32,
        kernel_size_2=3,
        pooling_size_2=2):
    """Constructs layers for a keras model based on a dict of model types."""
    model_layers = {
        'linear': [
            Flatten(),
            Dense(nclasses),
            Softmax()
        ],
        'dnn': [
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dense(nclasses),
            Softmax()
        ],
        'dnn_dropout': [
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dropout(dropout_rate),
            Dense(nclasses),
            Softmax()
        ],
        'dnn': [
            Conv2D(num_filters_1, kernel_size=kernel_size_1,
                   activation='relu', input_shape=(WIDTH, HEIGHT, 1)),
            MaxPooling2D(pooling_size_1),
            Conv2D(num_filters_2, kernel_size=kernel_size_2,
                   activation='relu'),
            MaxPooling2D(pooling_size_2),
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dropout(dropout_rate),
            Dense(nclasses),
            Softmax()
        ]
    }
    return model_layers[model_type]


def build_model(layers, output_dir):
    """Compiles keras model for image classification."""
    model = Sequential(layers)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


def train_and_evaluate(model, num_epochs, steps_per_epoch, output_dir):
    """Compiles keras model and loads data into it for training."""
    mnist = tf.keras.datasets.mnist.load_data()
    train_data = util.load_dataset(mnist)
    validation_data = util.load_dataset(mnist, training=False)

    callbacks = []
    if output_dir:
        tensorboard_callback = TensorBoard(log_dir=output_dir)
        callbacks = [tensorboard_callback]

    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=num_epochs,
        steps_per_epoch=steps_per_epoch,
        verbose=2,
        callbacks=callbacks)

    if output_dir:
        export_path = os.path.join(output_dir, 'keras_export')
        model.save(export_path, save_format='tf')

    return history


Overwriting mnistmodel/trainer/model.py


## Local Training

Now that we know that our models are working as expected, let's run it on the [Google Cloud AI Platform](https://cloud.google.com/ml-engine/docs/). We can run it as a python module locally first using the command line.

The below cell transfers some of our variables to the command line as well as create a job directory including a timestamp.

You can change the model_type to try out different models.

In [7]:
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
model_type = 'dnn'

os.environ["MODEL_TYPE"] = model_type
os.environ["JOB_DIR"] = "gs://{}/mnist_{}_{}/".format(
    BUCKET, model_type, current_time)
os.environ["JOB_NAME"] = "mnist_{}_{}".format(
    model_type, current_time)

The cell below runs the local version of the code. The epochs and steps_per_epoch flag can be changed to run for longer or shorther, as defined in our `mnistmodel/trainer/task.py` file.

In [8]:
%%bash
python3 -m mnistmodel.trainer.task \
    --job-dir=$JOB_DIR \
    --epochs=5 \
    --steps_per_epoch=50 \
    --model_type=$MODEL_TYPE

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
50/50 - 14s - loss: 1.1242 - accuracy: 0.6392 - val_loss: 0.3927 - val_accuracy: 0.8836
Epoch 2/5
50/50 - 8s - loss: 0.3596 - accuracy: 0.8808 - val_loss: 0.2167 - val_accuracy: 0.9332
Epoch 3/5
50/50 - 8s - loss: 0.2317 - accuracy: 0.9254 - val_loss: 0.1714 - val_accuracy: 0.9451
Epoch 4/5
50/50 - 8s - loss: 0.2138 - accuracy: 0.9346 - val_loss: 0.1187 - val_accuracy: 0.9659
Epoch 5/5
50/50 - 8s - loss: 0.1763 - accuracy: 0.9462 - val_loss: 0.0998 - val_accuracy: 0.9696


2020-08-15 18:19:26.919139: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2200135000 Hz
2020-08-15 18:19:26.919581: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564adbf49cb0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-08-15 18:19:26.919623: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-08-15 18:19:26.932035: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2020-08-15 18:19:27.774338: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-08-15 18:19:35.858467: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-08-15 18:19:38.407525: I tensorflow/core/profiler/rpc/client/save_profile.cc:168] Creating directory: gs://qwiklabs-gcp-00-02c9d8dfaa4

## Training on the cloud

Since we're using an unreleased version of TensorFlow on AI Platform, we can instead use a [Deep Learning Container](https://cloud.google.com/ai-platform/deep-learning-containers/docs/overview) in order to take advantage of libraries and applications not normally packaged with AI Platform. Below is a simple [Dockerlife](https://docs.docker.com/engine/reference/builder/) which copies our code to be used in a TF2 environment.

In [9]:
%%writefile mnistmodel/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-cpu
COPY mnistmodel/trainer /mnistmodel/trainer
ENTRYPOINT ["python3", "-m", "mnistmodel.trainer.task"]

Writing mnistmodel/Dockerfile


The below command builds the image and ships it off to Google Cloud so it can be used for AI Platform. When built, it will show up [here](http://console.cloud.google.com/gcr) with the name `mnistmodel`. ([Click here](https://console.cloud.google.com/cloud-build) to enable Cloud Build)

In [10]:
!docker build -f mnistmodel/Dockerfile -t $IMAGE_URI ./

Sending build context to Docker daemon  1.989MB
Step 1/3 : FROM gcr.io/deeplearning-platform-release/tf2-cpu
latest: Pulling from deeplearning-platform-release/tf2-cpu

167c320d: Pulling fs layer 
805ec7fd: Pulling fs layer 
d380e680: Pulling fs layer 
40b13098: Pulling fs layer 
a9266402: Pulling fs layer 
06dfac1d: Pulling fs layer 
427606e5: Pulling fs layer 
21666cec: Pulling fs layer 
b5d659ed: Pulling fs layer 
7556a8f1: Pulling fs layer 
b7e1223e: Pulling fs layer 
283ad922: Pulling fs layer 
fd48a5f3: Pulling fs layer 
d5dc04d2: Pulling fs layer 
4350f761: Pulling fs layer 
d0b9a318: Pulling fs layer 
f3738c10: Pulling fs layer 
557b1417: Pulling fs layer 
b94bc469: Pulling fs layer 
Digest: sha256:39dbd376df177650a465ac018d4e29782865b5e5600ad0376776c7b3de97d4cdDownloading     679B/8.285kB
Status: Downloaded newer image for gcr.io/deeplearning-platform-release/tf2-cpu:latest
 ---> 2a1788148fb1
Step 2/3 : COPY mnistmodel/trainer /mnistmodel/trainer
 ---> c5b4a240659c
Step 3/3 : 

In [11]:
!docker push $IMAGE_URI

The push refers to repository [gcr.io/qwiklabs-gcp-00-02c9d8dfaa4e/mnistmodel]

cc4073fd: Preparing 
ea7544d7: Preparing 
4908120a: Preparing 
32db29d1: Preparing 
c197f05c: Preparing 
59643149: Preparing 
f1bc8335: Preparing 
7fce4bef: Preparing 
25416dd1: Preparing 
9bef6670: Preparing 
397a1d54: Preparing 
5579b7a8: Preparing 
6c16a28a: Preparing 
f6c43556: Preparing 
17461bc6: Preparing 
474ff053: Preparing 
7a04bf9f: Preparing 
00d84994: Preparing 
52ea2c16: Preparing 
c9e5703f: Preparing 
7a04bf9f: Mounted from deeplearning-platform-release/tf2-cpu latest: digest: sha256:f9935f28a8d95157187d03786ffa0f258013aa2f34d3bf7d29a1ad3e00dc807e size: 4713


Finally, we can kickoff the [AI Platform training job](https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training). We can pass in our docker image using the `master-image-uri` flag.

In [12]:
%%bash
echo $JOB_DIR $REGION $JOB_NAME
gcloud ai-platform jobs submit training $JOB_NAME \
    --staging-bucket=gs://$BUCKET \
    --region=$REGION \
    --master-image-uri=$IMAGE_URI \
    --scale-tier=BASIC_GPU \
    --job-dir=$JOB_DIR \
    -- \
    --model_type=$MODEL_TYPE

gs://qwiklabs-gcp-00-02c9d8dfaa4e/mnist_dnn_200815_181914/ us-central1 mnist_dnn_200815_181914
jobId: mnist_dnn_200815_181914
state: QUEUED


Job [mnist_dnn_200815_181914] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe mnist_dnn_200815_181914

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs mnist_dnn_200815_181914


Can't wait to see the results? Run the code below and copy the output into the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true) to follow.

## Deploying and predicting with model

Once you have a model you're proud of, let's deploy it! All we need to do is give AI Platform the location of the model. Below uses the keras export path of the previous job, but `${JOB_DIR}keras_export/` can always be changed to a different path.

Uncomment the delete commands below if you are getting an "already exists error" and want to deploy a new model.

In [13]:
%%bash
MODEL_NAME="mnist"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=${JOB_DIR}keras_export/
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#yes | gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model ${MODEL_NAME} \
    --origin ${MODEL_LOCATION} \
    --framework tensorflow \
    --runtime-version=2.1

Deleting and deploying mnist dnn from gs://qwiklabs-gcp-00-02c9d8dfaa4e/mnist_dnn_200815_181914/keras_export/ ... this will take a few minutes


Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/qwiklabs-gcp-00-02c9d8dfaa4e/models/mnist].
Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......
...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Copyright 2020 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.